In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# configuration parameters
MASS = 140.87 #g
SIGMA_MASS = 0.49 #g
PERIOD = 0.804 #s
SIGMA_PERIOD = 0.010 #s
PULSATION = 7.815 #rad/s
SIGMA_PULSATION = 0.097 #rad/s
FREQUENCY = 1.244 #Hz
SIGMA_FREQUENCY = 0.015 #Hz
GAMMA = 0.00264 #1/s
SIGMA_GAMMA = 0.00021 #1/s

def max(array):
    
    max_value = array[0]
    
    for i in range(1, len(array)):
        
        if array[i] > max_value:
            
            max_value = array[i]
    
    return max_value

# functions which calculates the mean of the elements of a dictionary
def dic_mean(dic):
    
    n = 0
    sum = 0
    
    for key in dic:
        
        sum += dic[key]
        n += 1
    
    return (sum / n)

def chi_q(arr_obs, arr_th, c):
    
    chi = 0
    
    for i in range(len(arr_obs)):
        
        chi += ((arr_obs[i] - arr_th[i])**2) / arr_th[i]
    
    return (chi / (len(arr_obs) - c - 1))

# analysis function

def analysis(path_lorentz, lim_min):
    
    # import csv file for amplitudes
    lorentziana = pd.read_csv(path_lorentz, sep=';')
    
    # convert DataFrames to numpy arrays
    ampl = lorentziana['amplitude'].to_numpy()
    freq = lorentziana['frequency'].to_numpy()
    puls = 2 * np.pi * freq
    
    # evaluate the force acted on the spring
    amp_max = max(ampl)
    i_max, = np.where(ampl == amp_max)
    force = 2 * PULSATION * MASS * GAMMA * amp_max
    
    #defining lorentz's distribution
    def lorentz(omega):
        return (force / MASS) / (2 * PULSATION * np.sqrt((omega - PULSATION)**2 + GAMMA**2))
    
    # linspace for plotting
    omega = np.linspace(7.3, 8.1, 1000)
    
    # theoretical values of distribution
    ampl_theo = lorentz(puls)
    
    # normalization
    n = {}
    
    for i in range(1, len(ampl)):
            for j in range(i):
                if ((i - j) >= (lim_min-1) and (i_max < j or i_max > i)):
                    n[(j,i)] = sum(ampl_theo[j:i+1]) / sum(ampl[j:i+1])
    
    with open("analisi/forzato/normalization.log", 'w') as textfile:
        for key in n:
            print(key[0], "-", key[1],":\tN: ", n[key], file=textfile)
    
    chi = {}
    
    for key in n:
        chi[key] = chi_q(n[key] * ampl, ampl_theo, 1)
    
    with open("analisi/forzato/normalization_chi.log", 'w') as textfile:
        for key in chi:
            print(key[0], "-", key[1],":\tN: ", chi[key], file=textfile)
    
    key_norm = min(chi, key=chi.get)
    n_norm = n[key_norm]
    chi_norm = chi[key_norm]
    print("Normalization constant:", n_norm)
    print("Chi squared:", chi_norm)
    print("Interval:", key_norm)
    
    plt.plot(omega, lorentz(omega), label="th")
    plt.scatter(puls, ampl, marker='.', s=30, c='green', label="obs")
    plt.scatter(puls, n_norm*ampl, marker='.', s=30, c='red', label="obs norm")
    plt.xlabel("ω [rad/s]")
    plt.ylabel("A [mm]")
    plt.legend()
    plt.grid()
    plt.title("Lorentz's distribution")
    plt.savefig("analisi/forzato/plot_lorentz.png", dpi=1200)
    plt.close()

analysis('dati/forzato/lorentziana.csv', 10)

Normalization constant: 0.14763039208442752
Chi squared: 1.3820249859954694
Interval: (5, 14)
